## SDP application for BB84 protocol

We must first call the appropriate functions to implement the problem.

We have also defined an "operator" class to help us manipulate the fixtures in the big matrix T

The "reduce", "arrange" and "ReduceList" functions are use to simplifies the expression of the multiplication of multiple operators

The "NPAhierarchy" function builds the entire hierarchy at the level considered



In [1]:
import numpy as np
import picos
import cvxpy as cp
from scipy.stats import norm
import math
import sys
from pprint import pprint
from matplotlib import pyplot as plt

In [2]:
class operator:
    
    def __init__(self,h,m,o):  
        
        self.m = m  # measurement basis  
        self.o = o  # measurement projector, here we always consider outcome 0
        self.h = h  # Hilbert space  0 for Bob 1 for Eve
        
        
    def __repr__(self):
        rep = []
        i =0
        while i<(len(self.h)):
            if self.h[i]==0:
                rep = rep + ['B_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==1:
                rep = rep + ['E_({}|{})'.format(self.m[i], self.o[i])]
            elif self.h[i]==-1:
                rep = rep + ['Id']
            i = i+1
            
        return ''.join(rep)
    
    
    def __hash__(self):
        return hash((str(self.h), str(self.m), str(self.o))) 
    
                    
    def __mul__(self, other):
        return operator(self.h+other.h, self.m+other.m, self.o+other.o)
    
    
    __rmul__=__mul__
              
        
def reduce(OP):  # Simplifies the expression of the multiplication of multiple operators


    i=0
    l = len(OP.m)-1
    while i < l:
        ah = OP.h[i]
        ao = OP.o[i]
        am = OP.m[i]
        bh = OP.h[i+1]
        bo = OP.o[i+1]
        bm = OP.m[i+1]
        if ah != bh:
            if ah==1:  # Place the operator of Eve far right and those of Bob far left 
                OP.h[i], OP.h[i+1] = OP.h[i+1], OP.h[i]
                OP.o[i], OP.o[i+1] = OP.o[i+1], OP.o[i]
                OP.m[i], OP.m[i+1] = OP.m[i+1], OP.m[i]
            elif ah==-1:  # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i)
                (OP.o).pop(i)
                (OP.m).pop(i)
                l = l-1
                i = i-1
            elif bh==-1: # If the identity operator is found, we simplify the expresson and remove it
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
            else:
                pass
        elif ah==bh:
            if ah==1:  # 1 correspond to Eve
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
            elif ah==0:  # 0 correspond to Bob
                if am != bm:
                    pass
                else:
                    if ao==bo:
                        (OP.h).pop(i+1)
                        (OP.o).pop(i+1)
                        (OP.m).pop(i+1)
                        l = l-1
                        i = i-1   
                    else:
                        return 0
            elif ah==-1:
                (OP.h).pop(i+1)
                (OP.o).pop(i+1)
                (OP.m).pop(i+1)
                l = l-1
                i = i-1
                
  
        i = i+1
    return OP
        
        
def arrange(OPToReduce):
    if len(OPToReduce.m)==1:
        pass
    else:
        for i in range(len(OPToReduce.m)):
            reduce(OPToReduce)
    return reduce(OPToReduce)



def Transp(OP):
    if OP == 0:
        return 0
    else:
        h = OP.h
        o = OP.o
        m = OP.m
        T = operator(h[::-1],m[::-1],o[::-1])
        Transp = arrange(T)
        return T
    

def ReduceList(Mon):
    for i,m in enumerate(Mon):
        for j,n in enumerate(Mon):
            if i!=j:
                if hash(m)==hash(n):
                    Mon.pop(j)
                else:
                    pass
            else:
                pass
        
    return Mon



In [3]:
# mesure x outcome a , mesure 
def NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,lvl):
    Max = NOutcomeB*NMesureB+NOutcomeE*NMesureE +1
    
    if lvl == 1:
        Mon = [operator([-1],[0],[0])]
        MonT = [operator([-1],[0],[0])]
        for k in range(NMesureB):
            for i in range(NOutcomeB):
                Mon = Mon + [operator([0],[k],[i])]
                MonT = MonT + [operator([0],[k],[i])]
                
        for k in range(NMesureE):
            for i in range(NOutcomeE):
                Mon = Mon + [operator([1],[k],[i])]
                MonT = MonT + [operator([1],[k],[i])]
            
    elif lvl == 1.5:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for e in range(NMesureE):
            for eo in range(NOutcomeE):
                for k in range(NMesureB):
                    for i in range(NOutcomeB):
                        Mon = Mon + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
                        MonT = MonT + [(operator([0],[k],[i])*operator([1],[e],[eo]))]
        
    elif lvl == 2:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,1)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                #if i!=j:
                tot = arrange(n*m)
                totT = arrange(m*n)
                if tot !=0:
                    Mon = Mon + [tot]
                    MonT = MonT + [totT]
                else:
                    pass
                #else:
                 #   pass
    
    elif lvl == 3:
        Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,2)
        for i,m in enumerate(Mon[1:Max]):
            for j,n in enumerate(Mon[1:Max]):
                for k,p in enumerate(Mon[1:Max]):
                    #if i!=j and i!=k and k!=j:
                    tot = arrange(arrange(m*n)*p)
                    totT = arrange(arrange(p*n)*m)
                    if tot != 0:
                        Mon = Mon + [tot]
                        MonT = MonT + [totT]
                    else:
                        pass
                    #else:
                        #pass
        
        
    else:
        pass
    
    Mon = ReduceList(Mon)
    MonT = ReduceList(MonT)
    return Mon, MonT

In [4]:
def p0abB92CV(x,y,asqrt,theta,n):
    A = np.exp(-n*asqrt*np.sin(theta)**2*np.sin(np.pi*(x-y)/2)**2)
    return 1 - A

Once those function are defined, we can construct a dictionary of the different element in the big matrix T.

This allows us to reduce the number of variables for the problem.

In [6]:
NMesureB = 2
NOutcomeB = 1
NMesureE = 1
NOutcomeE = 1
LVLNPA = 2
states = 2
print('NPA LVL{}'.format(LVLNPA))
Mon, MonT = NPAhierarchy(NMesureB,NOutcomeB,NMesureE,NOutcomeE,LVLNPA)

#Mon = ReduceList(Mon)
#MonT = ReduceList(MonT)
            
        

MAX = len(Mon)
print('Sub-block: {}x{}'.format(MAX,MAX))
print('Big matrix: {}x{}'.format(states*MAX,states*MAX))
print(Mon)
print(MonT)
terms = {}
#termsT = {}

for i,m in enumerate(Mon):
    for j,n in enumerate(MonT):
        tot = arrange(n*m)
        tot_rep = hash(tot)
        if tot_rep in terms:
            terms[tot_rep].append((j,i))
        else:
            terms[tot_rep] = [(j,i)]

print("Number of different elements in a sub-block :", len(terms.keys()))
ndif = len(terms.keys())*((states*(states+1))//2)
print("Number of different variables in the problem :",ndif)

NPA LVL2
Sub-block: 8x8
Big matrix: 16x16
[Id, B_(0|0), B_(1|0), E_(0|0), B_(1|0)B_(0|0), B_(0|0)E_(0|0), B_(0|0)B_(1|0), B_(1|0)E_(0|0)]
[Id, B_(0|0), B_(1|0), E_(0|0), B_(0|0)B_(1|0), B_(0|0)E_(0|0), B_(1|0)B_(0|0), B_(1|0)E_(0|0)]
Number of different elements in a sub-block : 16
Number of different variables in the problem : 48


Once the separate elements have been identified, the big matrix T is built, making sure that it is Hermitian

We implement then the Gram constraints and the statistical constraints

# PICOS

In [ ]:
z = picos.RealVariable("z", ndif)

Matlist = []
for X in range(2):
    for Y in range(X,2):
        for k in terms.keys():
            B = np.zeros((2*MAX,2*MAX))
            if k ==0:
                Matlist.append(B)
            else:
                for i in terms[k]:
                    B[X*MAX+i[0],Y*MAX+i[1]] = 1    
                Matlist.append(B)
                
            
if len(Matlist) != ndif:
    print("Mismatch parameter number, shoud be :",ndif, " insted of :",len(Matlist))
    print("end program")
    sys.exit(1)
    
print("Number parameter match")
print("Summing matrix")
T = sum([(Matlist[i]*z[i] + Matlist[i].T*z.conj[i])for i in range(ndif)])
print("cheking hermitian")
if T.hermitian==True:
    print('∑[v_i * B_i + conj(v_i)* Transp(B_i)] is hermitian')
else:
    print("T is not hermitian!!")
        

#GguessBl = []
GguessAl = []
#GHBEl = []
GHAEl = []
GHBAl = []
#GKeyR = []
GKeyD = []
GARange = [x*0.2 for x in range(10*0, 9*1)][1:]
#ethaR = [x*0.05 for x in range(10*0, 10*2)][1:]#[x*0.01 for x in range(100,30,-10)]
#GARange = [x*0.01 for x in range(200,50,-10)]
ethaR = [x*0.05 for x in range(20,13,-1)]+[x*0.01 for x in range(64,49,-1)] +[0.4,0.3,0.2,0.1]#[x*0.01 for x in range(99,40,-5)]
theta = 0.2

for asqrt in GARange:
    print("${\alpha}$",asqrt)
    constraints = [T >> 0]
    guessBlC = []
    guessAlC = []
    HBElc = []
    HAElc = []
    HBAlc = []
    KeyRc = []
    KeyDc = []
    
#Gram Matrix
    P = picos.Problem()
    SDP = P.add_constraint( T >> 0)
    constraints = []
    Gram = np.array([[1,np.exp(-2*asqrt*np.sin(theta/2)**2)],
                            [np.exp(-2*asqrt*(np.sin(theta/2)**2)),1]])
    for X in range(states):
        for Y in range(states):
            constraints.append(T[MAX*X, MAX*Y] == Gram[X, Y])
    
    GramConst = P.add_list_of_constraints(constraints)
            
    for n in ethaR:
        print('transmission %', n*100 )
        constraints2 = []
        for X in range(states):
            for Y in range(states):
                constraints2.append(T[MAX*X+1+Y,MAX*X+1+Y] == p0abB92CV(X,Y,asqrt,theta,n))
        
        StatConst = P.add_list_of_constraints(constraints2)
                
        PGuessA = (T[2,3]+T[1,3]+T[MAX+1,MAX+1]-T[MAX+1,MAX+3]+T[MAX+2,MAX+2]-T[MAX+2,MAX+3])/4
    
        PGuessB = (T[1,3]+T[2,2]-T[2,3]+T[MAX+1,MAX+3]+T[MAX+2,MAX+2]-T[MAX+2,MAX+3])/4
        psucc = (T[MAX+1,MAX+1]+T[MAX+2,MAX+2]+T[1,1]+T[2,2])/4
                
        P.set_objective("max", PGuessA) #A MODIFIER EN FONCTION DE A OU B

        P.options.solver = "mosek"
        
        P.solve()
        print('PGuess value :',PGuessA.value)
        #guessB = PGuessB.value
        guessA = PGuessA.value
                
        #guessBlC.append(guessB)
        guessAlC.append(guessA/psucc.value)
        #pp = p0abB92CV(0,1,asqrt,theta,n)
        HBA = HBA_B92CV(n,asqrt,theta)
        #HBA = -(1-pp)*math.log2(1-pp)- pp*math.log2(pp)
        #HBA = 1/2
        
        #InfHPguessBc = -math.log2(guessB/psucc.value)
        InfHPguessAc = -math.log2(guessA/psucc.value)
        
        #HBElc.append(InfHPguessBc)
        HAElc.append(InfHPguessAc)
        HBAlc.append(HBA)
    
        #KeyRateReversc = (InfHPguessBc - HBA)*psucc.value
        KeyRateDirectc = (InfHPguessAc - HBA)*psucc.value
        
        #if KeyRateReversc < 0:
        #    KeyRc.append(0)
        #elif KeyRateReversc > 0:
        #    KeyRc.append(KeyRateReversc)
        if KeyRateDirectc < 0:
            KeyDc.append(0)
        elif KeyRateDirectc > 0:
            KeyDc.append(KeyRateDirectc)
        
        P.remove_constraint((2,))
    
        print("*****************")
    #GguessBl.append([asqrt,guessBlC])
    GguessAl.append([asqrt,guessAlC])
    #GHBEl.append([asqrt,HBElc])
    GHAEl.append([asqrt,HAElc])
    GHBAl.append([asqrt,HBAlc])
    #GKeyR.append([asqrt,KeyRc])
    GKeyD.append([asqrt,KeyDc])
    P.remove_constraint((1,))
    
print("DONE")